In [ ]:
%config IPCompleter.greedy=True
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import display
SHOW_GIFS = True

# History

The fathers of Particle Swarm Optimization technique are considered James Kennedy and Russel Eberhart, who present them in their paper "Particle Swarm Optimization" in 1995 [[1]]([#Bibliography]). Authors take inspiration in nature, concretely in bird flocking and fish schooling and in their simulations already implemented in that time. We can see the behaviours in the videos bellow.

In [ ]:
display.YouTubeVideo('0dskCpuxqtI?start=8', 560, 315)

In [ ]:
display.YouTubeVideo('Y-5ffl5_7AI?start=90', 560, 315)

If you play watch through the second video, you can see one of the attempt to fully understand how the fish schooling really behaves by tracking each fish. Although not very important to us, I think it's still interesting.

Let's return to the topic. As scientists found out, the behavior of swarms is most of the time based on very simple rules, but executed in distributed manner. For example, when fish in the swarm see food, it will swim toward it. Otherwise, the fish keeps the same direction as its neighboors. As some wish change their direction toward the food, the rest of the swarm would follow them. As a result, the whole swarm reach the food and, in case there is no competetion between individuals (enough food for each one), each fish feed itself. 

Although there is no explicit communication, the simple rules provide simple information sharing ability inside the flock, increasing the total gain of the group as a whole.

The authors of the paper tried some approaches both to optimize given function and to simulate flock most "lifely". I will start simple and improve the idea further. 

# Particles

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import cocoex
import functools
import functions as fn
import execution as exe
import plotting as plot
import inspect
import strategies
import utils
from PIL import Image
import os
import functools

First of all, we will have a set of particles scattered over the parameter space. We can for example let them move randomly around.

In [ ]:
definition = inspect.getsource(strategies.random_walk)
display.display(display.Markdown(f'```python\n{definition}\n```'))

In [ ]:
with fn.get_suite_wrapper(dimension=[2], function_indices=[7]) as suite:
    for function in suite:
        populations, values = exe.execute(function, strategies.random_walk, show_progress=True)
        
        plt.figure(figsize=(12,8))
        plot.plot_graph(values, plot.popfn_median())
        plt.show()
        
        if SHOW_GIFS:
            gif = plot.animate_movement(function, populations, show_progress=True)
            with open(os.path.join("main", "001.gif"), "wb") as f:
                f.write(gif);
            display.Markdown("![img](main/001.gif)")

In Particle Swarm Optimization, the positions are not updated directly, instead, we assign each particle a velocity. Each iteration we modify the velocity and let the velocity determine the next position. Again, we may initialized the velocity uniformly and then update it randomly each iteration. Formally, the position is updated as

$$
p_i(t+1) = p_t(t) + v_i(t+1) 
$$

Velocity update is what differs between approaches. In the previous case (with only random walks) the update is 

$$
v_i(t+1) = v_t(t) + N(0, \text{stepsize})
$$

Note that the velocity is the main reason, why we call the individuals particles, as velocity is usually connected with something with mass. Otherwise, we could call them just points or dots.

In [ ]:
definition = inspect.getsource(strategies.RandomVelocity)
display.display(display.Markdown(f'```python\n{definition}\n```'))

In [ ]:
with fn.get_suite_wrapper(dimension=[2], function_indices=[7]) as suite:
    for function in suite:
        populations, values = exe.execute(
            function, 
            strategies.RandomVelocity.execute, 
            initialization = strategies.RandomVelocity.init,
            show_progress=True)
        
        plt.figure(figsize=(12,8))
        plot.plot_graph(values, plot.popfn_median())
        plt.show()
        
        if SHOW_GIFS:
            gif = plot.animate_movement(function, populations, show_progress=True)
            with open(os.path.join("main", "002.gif"), "wb") as f:
                f.write(gif);
            display.Markdown("![img](main/002.gif)")

Note how the particles end up on the edge. Although I limit the position, so all particles stay within the area, it is not reflect in the in the velocity update. I will deal with it a bit later.

However, what is usually done is to decrease velocity a little bit every iteration, so the velocity can't "explode" too much. The parameter $w$ is there exactly for that purpose. We may set it to value between $(0.0, 1.0)$ to decrease velocity every step. As we will see, proposed constant by the paper is $w = \frac{1}{2 \cdot log(2)} \approx 0.721$.

In [ ]:
with fn.get_suite_wrapper(dimension=[2], function_indices=[7]) as suite:
    for function in suite:
        populations, values = exe.execute(
            function, 
            functools.partial(strategies.RandomVelocity.execute, w=1/(2*np.log(2))), 
            initialization = strategies.RandomVelocity.init,
            show_progress=True)
        
        plt.figure(figsize=(12,8))
        plot.plot_graph(values, plot.popfn_median())
        plt.show()
        
        if SHOW_GIFS:
            gif = plot.animate_movement(function, populations, show_progress=True)
            with open(os.path.join("main", "003.gif"), "wb") as f:
                f.write(gif);
            display.Markdown("![img](main/003.gif)")

# Comparison - differencial evolution

Previous two examples were great to show the main principes, however, the results were not very satysfying. I wanted some algorithm, that I could compare algorithms with. I choose Differential evolution [[2]](#Bibliography), that was published around the same time (1996) and thus is a good candidate for comparison. Moreover, from my experience, differential evolution works quiet well for continuous optimization.

In the differential evolution, new individuals are based on the following formula.
$$
m_n = \text{Uniform}(p_3, p_0 + F \cdot (p_1 - p_2), CR)
$$

Terms $p_0, \dots, p_3$ are parents. Firstly, the algorithm adds difference of two parents and then performs uniform mutation with probability $1 - CR$. Moreover, I used parental tournament selection to pick up parents for the next iteration.

In [ ]:
definition = inspect.getsource(strategies.differential_evolution)
display.display(display.Markdown(f'```python\n{definition}\n```'))

In [ ]:
with fn.get_suite_wrapper(dimension=[2, 5], function_indices=[7]) as suite:
    for function in suite:  # type: cocoex.Problem
        populations, values = exe.execute(
            function, 
            strategies.differential_evolution, 
            show_progress=True)

        plt.figure(figsize=(12,8))
        plot.plot_graph(values, plot.popfn_median())
        plt.title(f"Function f07 {function.dimension}D using differential evolution")
        plt.show()
        
        if SHOW_GIFS and function.dimension == 2:
            gif = plot.animate_movement(function, populations, show_progress=True)
            with open(os.path.join("main", "004.gif"), "wb") as f:
                f.write(gif);
            display.Markdown("![img](main/004.gif)")
            

        print(('Found' if function.final_target_hit else 'Not found') + " best solution", flush=True)
        print(f"Best evaluation: {function.best_observed_fvalue1}", flush=True)

As we can see, the algorithm is already doing something and although it doesn't find the optimal solution (that is $35.35$), it wasn't far away (the BBOB framework accepts solution around $10^{-9}$ of optimum). From now on, I will compare rest of the algorithms to this implementation of differential strategy.

# Taking best particle into account

In order to perform some optimization, we need to guide the particles somehow. The simplest approach is to pick up the best particle $b$ and move all other particles toward it. However, the particles shoudn't be fully deterministic. Therefore, I will introduce random weight to this direction, sampled randomly from uniform distribution between $0$ and $c$. Again, $c$ proposed by the paper is $c=0.5 + log(2) \approx 1.193$, so i will stick to this value. Formally, weight update is

$$
v_i = w \cdot v_t + \text{Unif}(0,c)\cdot(b - p_i) + N(0, \textit{stepsize})
$$

We can thing about the best perticle in a swarm as about some swarm knowledge about the problem.

In [ ]:
definition = inspect.getsource(strategies.FollowBest)
display.display(display.Markdown(f'```python\n{definition}\n```'))

In [ ]:
with fn.get_suite_wrapper(dimension=[2, 5], function_indices=[7]) as suite:
    for function in suite:  # type: cocoex.Problem
        populations, values = exe.execute_multiple(
            function, 
            strategies.FollowBest.execute,
            initialization=strategies.FollowBest.init,
            show_progress=True)
        print(('Found' if function.final_target_hit else 'Not found') + " best solution", flush=True)
        print(f"Best evaluation: {function.best_observed_fvalue1}", flush=True)
        eva_population, eva_values = exe.execute_multiple(function,strategies.differential_evolution)

        plt.figure(figsize=(12,8))
        plt.title(function.name)
        plot.plot_aggregated(values, plot.popfn_median(), plot.aggfn_median(), label="Follow best", c='r')
        plot.plot_aggregated(eva_values, plot.popfn_median(), plot.aggfn_median(), label="Differential evolution", c='b')
        plt.legend()
        plt.show()
        
        if SHOW_GIFS and function.dimension == 2:
            gif = plot.animate_movement(function, populations[0], show_progress=True)
            with open(os.path.join("main", "005.gif"), "wb") as f:
                f.write(gif);
            display.Markdown("![img](main/005.gif)")

The approach has one major drawback. Because all the particles are attracted by the best one, they are all moving to the same position. That can easily cause stick in the local optima, as the best particle doesn't  need to be close to the global one. It is easy to demonstrace using function with many local optima - for example `f24`.

In [ ]:
with fn.get_suite_wrapper(dimension=[2, 5], function_indices=[24]) as suite:
    for function in suite:  # type: cocoex.Problem
        populations, values = exe.execute_multiple(
            function, 
            strategies.FollowBest.execute,
            initialization=strategies.FollowBest.init,
            show_progress=True)
        print(('Found' if function.final_target_hit else 'Not found') + " best solution", flush=True)
        print(f"Best evaluation: {function.best_observed_fvalue1}", flush=True)
        eva_population, eva_values = exe.execute_multiple(function,strategies.differential_evolution)

        plt.figure(figsize=(12,8))
        plt.title(function.name)
        plot.plot_aggregated(values, plot.popfn_median(), plot.aggfn_median(), label="Follow best", c='r')
        plot.plot_aggregated(values, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='r')
        plot.plot_aggregated(eva_values, plot.popfn_median(), plot.aggfn_median(), label="Differential evolution", c='b')
        plot.plot_aggregated(eva_values, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='b')
        plt.legend()
        plt.show()
        
        if SHOW_GIFS and function.dimension == 2:
            gif = plot.animate_movement(function, populations[0], show_progress=True)
            with open(os.path.join("main", "006.gif"), "wb") as f:
                f.write(gif);
            display.Markdown("![img](main/006.gif)")

Depending on the run, particles get stuck around $\left[2,-2\right]$ or $\left[-4, 4\right]$.In any case, they never converg and are moving around chaotically (that can be caused by the random movement as well). This can be seen from the plots as well, as the differential evolution is in order of magnitude better.

On other hand, this allows particles to evolve over the whole time, compare to the differential evolution, which reached value around $97$ (for `f07`) respectivelly $117$ (for `f24`) and stucked there.

# Neighborhood

This reasoning led to introduce neighborhood. Instead of following best particle from the whole swarm, each particle has some neighbors, with which it compares it's value. The definition of neighborhood (sometimes called topology) is not defined preciselly and there are many approaches.

Firstly, the topology may change after each iteration or may remain static the whole time. In the first approach, each particle will eventually obtain information from all other particles - and thus can compare it's best known value with all the particles. Constant $K$ denotes how many particles inform each iteration and was set t $K=3$ in the paper. Because $K$ is quiet small, the problem is getting stuck in the local optimum is significantly reduced. In the second approach, the particle doesn't know about rest of the search space, unless some of it's neighbor reach it. This prevent swarm to get stuck in the local optima, but reduce amount of information each particle obtain a lot.

Next we need to decide how the topology would look like. First (and deterministic) approach proposed by the paper was called "ring topology". When we index particles, the ring topology for particle $i$ is $\left\{i+1\ mod(N), i+2\ mod(N), \dots, i+K\ mod(N)\right\}$ where $N$ is number of particles. Another proposition was "adaptive random topology", where each particle picks three other particles at random and inform them about it's value (note that this topology in its static version is same as the previous one). Another topology is "von Neumann" topology, that you can see bellow (I will not discuss it here anymore, as it is just modification of the ring topology). Finally, we may simple use $K$ nearest neighboors and inform them (or take information from them).

<a href="https://www.researchgate.net/figure/von-Neumann-neighborhood-structure-for-PSO_fig3_220742792"><img src="https://www.researchgate.net/profile/Enrique_Villa_Diharce/publication/220742792/figure/fig3/AS:472200114380802@1489592688251/von-Neumann-neighborhood-structure-for-PSO.png" alt="von Neumann neighborhood structure for PSO"/></a>

<center>Figure 3: von Neumann neighborhood structure for PSO <a href="#Bibliography">[3]</a></center>

I will show all the topologies and compare them. Let's start with the ring topology.

In [ ]:
definition = inspect.getsource(strategies.RingTopology)
display.display(display.Markdown(f'```python\n{definition}\n```'))

In [ ]:
with fn.get_suite_wrapper(dimension=[2, 5], function_indices=[7]) as suite:
    for function in suite:  # type: cocoex.Problem
        populations, values = exe.execute_multiple(
            function, 
            strategies.RingTopology.execute,
            initialization=strategies.RingTopology.init,
            show_progress=True)
        print(('Found' if function.final_target_hit else 'Not found') + " best solution", flush=True)
        print(f"Best evaluation: {function.best_observed_fvalue1}", flush=True)
        eva_population, eva_values = exe.execute_multiple(function,strategies.differential_evolution)

        plt.figure(figsize=(12,8))
        plt.title(f"Function f07 {function.dimension} using ring topology")
        plot.plot_aggregated(values, plot.popfn_median(), plot.aggfn_median(), label="Ring topology", c='r')
        plot.plot_aggregated(values, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='r')
        plot.plot_aggregated(eva_values, plot.popfn_median(), plot.aggfn_median(), label="Differential evolution", c='b')
        plot.plot_aggregated(eva_values, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='b')
        plt.legend()
        plt.show()
        
        if SHOW_GIFS and function.dimension == 2:
            gif = plot.animate_movement(function, populations[0], show_progress=True, title="Function f07 using ring topology")
            with open(os.path.join("main", "007.gif"), "wb") as f:
                f.write(gif);
            display.Markdown("![img](main/007.gif)")

The results are about worse than following the best particle in the swarm. The reason (what I believe) is the information throughput is very small. In order to particle know about better position, some of it's neighbors needs to reach it first - the neighbor can't tell it directly, even if it knows about it. Therefore, behavior is more chaotic and as we can see from the gif, the particles doesn't converge at all.

Now, let's try adaptive random topology.

In [ ]:
definition = inspect.getsource(strategies.RandomTopology)
display.display(display.Markdown(f'```python\n{definition}\n```'))

In [ ]:
with fn.get_suite_wrapper(dimension=[2, 5], function_indices=[7]) as suite:
    for function in suite:  # type: cocoex.Problem
        populations, values = exe.execute_multiple(
            function, 
            strategies.RandomTopology.execute,
            initialization=strategies.RandomTopology.init,
            show_progress=True)
        print(('Found' if function.final_target_hit else 'Not found') + " best solution", flush=True)
        print(f"Best evaluation: {function.best_observed_fvalue1}", flush=True)
        eva_population, eva_values = exe.execute_multiple(function,strategies.differential_evolution)

        plt.figure(figsize=(12,8))
        plt.title(f"Function f07 {function.dimension} using adaptive random topology")
        plot.plot_aggregated(values, plot.popfn_median(), plot.aggfn_median(), label="Random topology", c='r')
        plot.plot_aggregated(values, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='r')
        plot.plot_aggregated(eva_values, plot.popfn_median(), plot.aggfn_median(), label="Differential evolution", c='b')
        plot.plot_aggregated(eva_values, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='b')
        plt.legend()
        plt.show()
        
        if SHOW_GIFS and function.dimension == 2:
            gif = plot.animate_movement(function, populations[0], show_progress=True, title="Function f07 using adaptive random topology")
            with open(os.path.join("main", "008.gif"), "wb") as f:
                f.write(gif);
            display.Markdown("![img](main/008.gif)")

Using the same constants, the results are much better. There is higher probabilily, that the particle would know about better position much sooner and therefore the swarm created cluster around the global optimum, althought it seems quiet hard to converge there. I believe it is caused by the random steps, as when I removed it, algorithm performed better.

Note that the original reason for random steps was to add some randomness into the process. As we are picking up neigbors randomly this time, we don't need random steps anymore.

Finally, let's look at the nearest topology.

In [ ]:
definition = inspect.getsource(strategies.NearestTopology)
display.display(display.Markdown(f'```python\n{definition}\n```'))

In [ ]:
with fn.get_suite_wrapper(dimension=[2, 5], function_indices=[7]) as suite:
    for function in suite:  # type: cocoex.Problem
        populations, values = exe.execute_multiple(
            function, 
            strategies.NearestTopology.execute,
            initialization=strategies.NearestTopology.init,
            show_progress=True)
        print(('Found' if function.final_target_hit else 'Not found') + " best solution", flush=True)
        print(f"Best evaluation: {function.best_observed_fvalue1}", flush=True)
        eva_population, eva_values = exe.execute_multiple(function,strategies.differential_evolution)

        plt.figure(figsize=(12,8))
        plt.title(f"Function f07 {function.dimension} using nearest topology")
        plot.plot_aggregated(values, plot.popfn_median(), plot.aggfn_median(), label="Random topology", c='r')
        plot.plot_aggregated(values, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='r')
        plot.plot_aggregated(eva_values, plot.popfn_median(), plot.aggfn_median(), label="Differential evolution", c='b')
        plot.plot_aggregated(eva_values, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='b')
        plt.legend()
        plt.show()
        
        if SHOW_GIFS and function.dimension == 2:
            gif = plot.animate_movement(function, populations[0], show_progress=True, title="Function f07 using ring topology")
            with open(os.path.join("main", "009.gif"), "wb") as f:
                f.write(gif);
            display.Markdown("![img](main/009.gif)")

Although the algorithm seems to perform quiet well, from the movement gif is apparent main disadvantage - particles can create clusters away from the optimum, where they support each other (as they are neighbors). This approach can easily converge to local optima only.

Let's see how would the approaches handles `f24` with many local optima.

In [ ]:
with fn.get_suite_wrapper(dimension=[2, 5], function_indices=[24]) as suite:
    for function_id in suite.ids():  # type: cocoex.Problem
        suite.reset(); function = suite.get_problem(function_id)
        pop_ring, val_ring = exe.execute_multiple(function,strategies.RingTopology.execute,initialization=strategies.RingTopology.init,show_progress=True)
        print(('Found' if function.final_target_hit else 'Not found') + " best solution using ring topology", flush=True)
        print(f"Best evaluation: {function.best_observed_fvalue1}", flush=True)
        suite.reset(); function = suite.get_problem(function_id)
        pop_rand, val_rand = exe.execute_multiple(function,strategies.RandomTopology.execute,initialization=strategies.RandomTopology.init,show_progress=True)
        print(('Found' if function.final_target_hit else 'Not found') + " best solution using random topology", flush=True)
        print(f"Best evaluation: {function.best_observed_fvalue1}", flush=True)
        suite.reset(); function = suite.get_problem(function_id)
        pop_near, val_near = exe.execute_multiple(function,strategies.NearestTopology.execute,initialization=strategies.NearestTopology.init,show_progress=True)
        print(('Found' if function.final_target_hit else 'Not found') + " best solution using nearest topology", flush=True)
        print(f"Best evaluation: {function.best_observed_fvalue1}", flush=True)
        suite.reset(); function = suite.get_problem(function_id)
        pop_eva, val_eva = exe.execute_multiple(function,strategies.differential_evolution, show_progress=True)
        print(('Found' if function.final_target_hit else 'Not found') + " best solution using differential evolution", flush=True)
        print(f"Best evaluation: {function.best_observed_fvalue1}", flush=True)

        plt.figure(figsize=(12,8));
        plt.title(function.name);
        plot.plot_aggregated(val_ring, plot.popfn_median(), plot.aggfn_median(), label="Ring topology", c='r')
        plot.plot_aggregated(val_ring, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='r')
        plot.plot_aggregated(val_rand, plot.popfn_median(), plot.aggfn_median(), label="Random topology", c='g')
        plot.plot_aggregated(val_rand, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='g')
        plot.plot_aggregated(val_near, plot.popfn_median(), plot.aggfn_median(), label="Nearest topology", c='#471c7c')
        plot.plot_aggregated(val_near, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='#471c7c')
        plot.plot_aggregated(val_eva, plot.popfn_median(), plot.aggfn_median(), label="Differential evolution", c='b')
        plot.plot_aggregated(val_eva, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='b')
        plt.legend();
        plt.show()
        
        if SHOW_GIFS and function.dimension == 2:
            gif = plot.animate_movement(function, pop_ring[0], show_progress=True, title="Function f24 using ring topology")
            with open(os.path.join("main", "010.gif"), "wb") as f:
                f.write(gif);
            display.Markdown("![img](main/010.gif)")
            gif = plot.animate_movement(function, pop_rand[0], show_progress=True, title="Function f24 using random topology")
            with open(os.path.join("main", "011.gif"), "wb") as f:
                f.write(gif);
            display.Markdown("![img](main/011.gif)")
            gif = plot.animate_movement(function, pop_near[0], show_progress=True, title="Function f24 using nearest topology")
            with open(os.path.join("main", "012.gif"), "wb") as f:
                f.write(gif);
            display.Markdown("![img](main/012.gif)")
            gif = plot.animate_movement(function, pop_eva[0], show_progress=True, title="Function f24 using differential evolution")
            with open(os.path.join("main", "013.gif"), "wb") as f:
                f.write(gif);
            display.Markdown("![img](main/013.gif)")

# PSO 2006 - memory

Problem with the approaches above were, that the particles take into account into current state of the swarm. Let's take for example particle at good spot and it's neighborhood of very bad particles. Although all particles in the neighborhood have worse value than the current particle, it will try to move into worse position. That's not very fortune and with small modification in the source code we could fix this issue, however I will take different approach and allow particles to remember best global position $b_g$ (best position obtained from other particles) and best local position $b_l$ (best position of the particle itself) over the whole evaluation.

This brings us to the algorithm `Standard2006` or *SPSO 2006* (Standard Particle Swarm Optimisation 2006) [[4]](#Bibliography). Purpose of this paper was to standardize different implementations of Particle Swarm Optimization algorithm and mainly provide reference implementation that other algorithms can be compared with. The *SPSO 2006* algorithm is in fact the same algorithm, that were proposed by the "Particle Swarm Optimisation"[[1]](#Bibliography) paper in 1995.

The implementation uses adaptive random topology and the velocity update is 

$$
v_i = w \cdot v_i + \text{Unif}(0,c)\cdot(b_g - p_i) + \text{Unif}(0,c)\cdot(b_l - p_i)
$$

Note that I removed the random step term, as the randomnes in sampling from uniform distribution is enough.

To follow the standard implementation, I changed the initialization of velocities to $v_i(0)=\frac{\text{Unif}(min, max) - p_i(0)}{2}$. Original paper also uses synchronous updates (looping over all the particles sequentially). I decided implement the algorithm in an asynchronous manner - it better reflects our idea of "moving particles" and `numpy` library allows to vectorize the instructions, so the improvement in performance is significant.

In [ ]:
definition = inspect.getsource(strategies.Standard2006)
display.display(display.Markdown(f'```python\n{definition}\n```'))

In [ ]:
with fn.get_suite_wrapper(dimension=[2, 5], function_indices=[7, 24]) as suite:
    gif_counter = 14
    for function in suite:  # type: cocoex.Problem
        populations, values = exe.execute_multiple(
            function, 
            strategies.Standard2006.execute,
            initialization=strategies.Standard2006.init,
            show_progress=True)
        print(('Found' if function.final_target_hit else 'Not found') + " best solution", flush=True)
        print(f"Best evaluation: {function.best_observed_fvalue1}", flush=True)
        eva_population, eva_values = exe.execute_multiple(function,strategies.differential_evolution)

        plt.figure(figsize=(12,8))
        plt.title(function.name)
        plot.plot_aggregated(values, plot.popfn_median(), plot.aggfn_median(), label="Standard 2006", c='r')
        plot.plot_aggregated(values, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='r')
        plot.plot_aggregated(eva_values, plot.popfn_median(), plot.aggfn_median(), label="Differential evolution", c='b')
        plot.plot_aggregated(eva_values, plot.popfn_min(), plot.aggfn_median(), linestyle='--', c='b')
        plt.legend()
        plt.show()
        
        if SHOW_GIFS and function.dimension == 2:
            gif = plot.animate_movement(function, populations[0], show_progress=True)
            with open(os.path.join("main", f"{gif_counter:03d}.gif"), "wb") as f:
                f.write(gif);
            display.Markdown(f"![img](main/{gif_counter:03d}.gif)")
            gif_counter = gif_counter + 1

Finally, we have better algorithm than differencial evolution. Although median of the standard 2006 algorithm is worse, at least some particles reach lower (i.e. better) values. The problem of differential evolution is that the individuals are "static" - when all individuals reach some region, there is no way to get out of it. That is not necesarry true for the particles - even if they reach the best known optimum, they usually have some velocity left. That cause them to continue according to velocity and allow them to explore parameter space around. There is very low probability that particle would stop moving.

Before moving on, there is *SPSO 2007* algorithm that adds just cosmetic updates to this algorithm. Mainly, its permutate the particles before each iteration (as it uses synchronous updates). I decided for asynchronous updates, so this change doesn't bother us. Another change is usage of KISS pseudorandom generator - again something, that doesn't bother me. Therefore I will not be talking about *SPSO 2007* anymore.

# PSO 2011

# Bibliography

\[1\] J. Kennedy and R. Eberhart, "Particle swarm optimization," Proceedings of ICNN'95 - International Conference on Neural Networks, Perth, WA, Australia, 1995, pp. 1942-1948 vol.4.

\[2 \] R. Storn, "On the usage of differential evolution for function optimization," Proceedings of North American Fuzzy Information Processing, Berkeley, CA, USA, 1996, pp. 519-523.

\[3 \] Munoz-Zavala, Angel & Hernandez-Aguirre, Arturo & Villa Diharce, Enrique. (2009). The singly-linked ring topology for the particle swarm optimization algorithm. 65-72. 10.1145/1569901.1569911. 

\[4 \] Maurice Clerc. Standard Particle Swarm Optimisation. 2012. ffhal-00764996